In [ ]:
!pip install openai
!pip install datasets
!pip install transformers peft accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
from peft import LoraConfig, get_peft_model
import openai
from peft import LoraConfig, get_peft_model
from openai import OpenAI
import os
import pandas as pd
import numpy as np
import torch
trainer.evaluate()

#from datasets import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    print("Adding pad_token to tokenizer")
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [ ]:
df = pd.read_csv("IMDB Dataset.csv")


ParserError: Error tokenizing data. C error: EOF inside string starting at row 7089

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df["sentiment"].values

array(['positive', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [ ]:
df['sentiment'] = df['sentiment'].replace({'positive':0, 'negative':1})

<ipython-input-148-33ae0f38cae8>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive':0, 'negative':1})


In [ ]:
df["sentiment"].values

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True, padding="max_length", max_length=700)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
#reviews = tokenized_dataset["review"]
tokenized_dataset = tokenized_dataset.remove_columns(["review"])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])

In [ ]:
tokenized_dataset = tokenized_dataset.rename_column("sentiment", "labels")

In [ ]:
print(tokenized_dataset['input_ids'].shape)
print(tokenized_dataset['attention_mask'].shape)
print(tokenized_dataset['labels'].shape)

torch.Size([50000, 700])
torch.Size([50000, 700])
torch.Size([50000])


In [ ]:
train_test = tokenized_dataset.train_test_split(test_size=0.2)  # 80% train, 20% test
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

In [ ]:
sentiment_scores = torch.tensor(df["sentiment"].values)  # Assuming this is your sentiment column

# Add sentiment scores to eval_dataset if needed
#eval_dataset = eval_dataset.add_column("sentiment", sentiment_scores.tolist())

In [ ]:
sentiment_scores

tensor([0, 0, 0,  ..., 1, 1, 1])

In [ ]:
print(train_dataset[0])

NameError: name 'train_dataset' is not defined

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none"
)


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,736,256 || trainable%: 0.2364


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [ ]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 40000
})

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora-finetuned-gpt2",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    evaluation_strategy="steps",
    metric_for_best_model="accuracy",
    remove_unused_columns=False
)

# Prepare your dataset
# Ensure your datasets are prepared as `train_dataset` and `eval_dataset`

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,  # Replace with your training dataset
    eval_dataset= eval_dataset      # Replace with your evaluation dataset
)

# Fine-tune the model with LoRA
trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'pad_to_max_length'